## UCI_Bank_Marketing Dataset - Classification problem
### Multi-layer Perceptron approach

### Libraries

In [210]:
import tensorflow as tf
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import MinMaxScaler

### Load the dataset

In [211]:
#Read the dataset
Banco = pd.read_csv('C:/Users/campus/Downloads/TensorFlow_Tutorials/bank-additional-full.csv', sep=';')

In [212]:
#Replace the 'yes' and 'no' in the target variable to 'Subscriber' and 'non-subscriber'
#Doing so will help us during the train-test split phase
Banco['y'].replace('yes','Subscriber', inplace=True)
Banco['y'].replace('no', 'Non-subscriber', inplace=True)

In [213]:
#Create two seperate dataframes - categorial and numeric
Banco_cat = Banco.select_dtypes(exclude=[np.number])
Banco_num = Banco.select_dtypes(include=[np.number])

### Scale the variables

In [214]:
#Scale the numerical variables
min_max_scaler = MinMaxScaler()

Banco_num = pd.DataFrame(min_max_scaler.fit_transform(Banco_num), columns=Banco_num.columns)

In [215]:
Banco_mod = Banco_num

### Get dummies - categorical variables

In [216]:
#Get dummies for all categorical variables and concatenate the dataframes
for col in Banco_cat.columns:
        Banco_mod = pd.concat([Banco_mod,pd.get_dummies(Banco_cat[col])], axis=1)

In [217]:
#Check the concatenated dataframe
Banco_mod.head(2)

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,fri,mon,thu,tue,wed,failure,nonexistent,success,Non-subscriber,Subscriber
0,0.481481,0.053070,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735,...,0,1,0,0,0,0,1,0,1,0
1,0.493827,0.030297,0.0,1.0,0.0,0.9375,0.698753,0.60251,0.957379,0.859735,...,0,1,0,0,0,0,1,0,1,0


In [218]:
Banco_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 65 columns):
age                    41188 non-null float64
duration               41188 non-null float64
campaign               41188 non-null float64
pdays                  41188 non-null float64
previous               41188 non-null float64
emp.var.rate           41188 non-null float64
cons.price.idx         41188 non-null float64
cons.conf.idx          41188 non-null float64
euribor3m              41188 non-null float64
nr.employed            41188 non-null float64
admin.                 41188 non-null uint8
blue-collar            41188 non-null uint8
entrepreneur           41188 non-null uint8
housemaid              41188 non-null uint8
management             41188 non-null uint8
retired                41188 non-null uint8
self-employed          41188 non-null uint8
services               41188 non-null uint8
student                41188 non-null uint8
technician             41188 non-nu

### Train-test split

In [219]:
# Get X and y data - Drop the one hot encoded target classes from X
# Get the values of X and y as numpy arrays instead of pandas series
X = Banco_mod.drop(['Subscriber','Non-subscriber'],axis=1).values
y = Banco_mod[['Subscriber', 'Non-subscriber']].values           

In [220]:
# Train-test split - Considering 90:10 ratio for train and test respectively
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.1,random_state = 101)

### Neural Network 

In [221]:
# Network parameters
# randomly assign neurons in each layer
n_input = Banco_mod.shape[1] - 2   # Input size (There are only 63 input variables)
n_hidden_1 = 100                   # 1st layer
n_hidden_2 = 200                   # 2nd layer
n_hidden_3 = 200                   # 3rd layer
n_hidden_4 = 50                    # 4th layer
n_classes = 2                      # output m classes

# X and y - tf placeholders
X = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
dropout_keep_prob = tf.placeholder(tf.float32)

In [222]:
#Define weights and biases for each layer as dictionaries
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1],stddev=0.1)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],stddev=0.1)),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3],stddev=0.1)),
    'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4],stddev=0.1)),
    'out': tf.Variable(tf.random_normal([n_hidden_4, n_classes],stddev=0.1)),                                   
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'b4': tf.Variable(tf.random_normal([n_hidden_4])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

### Multi-layer perceptron Model

In [223]:
#Define Multi-layer perceptron model
def mlp(_X, _weights, _biases, dropout_keep_prob):
    
    #Layer 1
    layer1 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])), dropout_keep_prob)
    
    #Layer 2
    layer2 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(layer1, _weights['h2']), _biases['b2'])), dropout_keep_prob)
    
    #layer 3
    layer3 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(layer2, _weights['h3']), _biases['b3'])), dropout_keep_prob)
    
    #layer 4
    layer4 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(layer3, _weights['h4']), _biases['b4'])), dropout_keep_prob)
    
    #Output layer
    out = tf.nn.tanh(tf.add(tf.matmul(layer4, _weights['out']), _biases['out']))
    
    return out

### __Model__

In [237]:
#Neural Net Parameters
LEARNING_RATE = 0.001                                 # Learning rate
TRAINING_EPOCHS = 500                                # Number of epochs
BATCH_SIZE = 100                                      # Batch size

In [225]:
# Build model to predict the target classes
pred = mlp(X, weights, biases, dropout_keep_prob)

In [226]:
# Loss/cost function 
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y)) 

# Optimizer to minimise the loss/cost - using Adam Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE).minimize(cost)

In [227]:
# Accuracy of the predicted value
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Train the model

In [242]:
# Initialize variables
initialise = tf.initialize_all_variables()

# Launch session
sess = tf.Session()
sess.run(initialise)

In [238]:
# Training loop
print("Epoch\t\tCost\tAccuracy")
print("---------------------------------")
for epoch in range(TRAINING_EPOCHS):
    avg_cost = 0.
    total_batch = int(X_train.shape[0] / BATCH_SIZE)
    
    # Loop over all batches
    for i in range(total_batch):
        
        # Choose random records for each training batch
        choice = np.random.randint(X_train.shape[0], size = BATCH_SIZE)
        batch_xs = X_train[choice, :]
        batch_ys = y_train[choice, :]
        
        # Fit using batched data
        sess.run(optimizer, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob: 0.9})
        
        # Calculate average cost 
        avg_cost += sess.run(cost, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob:1.})/total_batch
    
    # Display progress after every 20 epochs
    
    if epoch % 20 == 0:
        
        training_acc = sess.run(accuracy, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob:1.})
        print ("%04d/%04d\t%.4f\t%.3f" % (epoch, TRAINING_EPOCHS, avg_cost,training_acc))
        

Epoch		Cost	Accuracy
---------------------------------
0000/0500	0.3251	0.940
0020/0500	0.2555	0.860
0040/0500	0.2467	0.930
0060/0500	0.2268	0.950
0080/0500	0.2104	0.960
0100/0500	0.1966	0.970
0120/0500	0.1861	0.980
0140/0500	0.1825	0.960
0160/0500	0.1723	0.990
0180/0500	0.1711	0.980
0200/0500	0.1699	1.000
0220/0500	0.1679	0.980
0240/0500	0.1627	1.000
0260/0500	0.1639	0.990
0280/0500	0.1641	0.970
0300/0500	0.1637	1.000
0320/0500	0.1626	0.980
0340/0500	0.1629	0.950
0360/0500	0.1586	0.960
0380/0500	0.1609	0.970
0400/0500	0.1601	0.950
0420/0500	0.1578	0.990
0440/0500	0.1597	0.980
0460/0500	0.1556	1.000
0480/0500	0.1588	0.980


### Validation on Test data

In [243]:
# Testing

testing_acc = sess.run(accuracy, feed_dict={X: X_test, y: y_test, dropout_keep_prob:1.})
print ("Test accuracy: %.4f" % (testing_acc))

Test accuracy: 0.1194


In [240]:
sess.close()

### Summary

An accuracy of __89.6%__ for 500 epochs. Accuracy is lesser than XGBoost model, however, it can improved when training epochs are increased 